In [23]:
import msgpack
import spacy
import textacy
from spacy.language import Language as SpacyLang
from spacy.tokens.doc import Doc as SpacyDoc
from textacy import cache
from textacy import compat
from textacy.io.utils import open_sesame
from pympler import asizeof
import humanfriendly
import os, sys, re, gc
import psutil

def remove_whitespace_entities(doc):
    doc.ents = [ e for e in doc.ents if not e.text.isspace() ]
    return doc

def keep_hyphen_tokenizer(nlp):
    infix_re = re.compile(r'''[.\,\?\:\;\...\‘\’\`\“\”\"\'~]''')
    prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)
    
def read_spacy_docs(fname, lang):
    vocab = lang.vocab
    docs = []
    with open_sesame(fname, mode='rb') as f:

        unpacker = msgpack.Unpacker(f, encoding='UTF-8')

        for msg in unpacker:

            if "user_data_keys" in msg:

                user_data_keys = msgpack.loads(msg["user_data_keys"], use_list=False, encoding='utf-8')
                for encoding in ['utf-8', 'latin1']:
                    try:
                        user_data_values = msgpack.loads(msg["user_data_values"], encoding=encoding)
                    except:
                        if encoding == 'latin1': raise

                user_data = { key: value for key, value in compat.zip_(user_data_keys, user_data_values)}
            else:
                print('error')
                user_data = None

            text = msg["text"]
            attrs = msg["array_body"]
            words = []
            spaces = []
            start = 0
            for i in compat.range_(attrs.shape[0]):
                end = start + int(attrs[i, 0])
                has_space = int(attrs[i, 1])
                words.append(text[start: end])
                spaces.append(bool(has_space))
                start = end + has_space

            spacy_doc = SpacyDoc(vocab, words=words, spaces=spaces, user_data=user_data)
            spacy_doc = spacy_doc.from_array(msg["array_head"][2:], attrs[:, 2:])
            if "sentiment" in msg:
                spacy_doc.sentiment = msg["sentiment"]
            if "tensor" in msg:
                spacy_doc.tensor = msg["tensor"]

#            print(
#                'msg: ' + humanfriendly.format_size(asizeof.asizeof(msg)),
#                'words: ' + humanfriendly.format_size(asizeof.asizeof(words)),
#                'array_head: ' + humanfriendly.format_size(asizeof.asizeof(msg["array_head"])),
#                'array_body: ' + humanfriendly.format_size(asizeof.asizeof(msg["array_body"])),
#                'spacy_doc: ', asizeof.asizeof(spacy_doc)
#            )

            docs.append(spacy_doc)
    return docs

p = psutil.Process(os.getpid())

print('MEM: ' + humanfriendly.format_size(p.memory_info().rss))

if 'corpus' in globals(): del corpus
if 'nlp' in globals(): del nlp
if 'docs' in globals(): del docs

gc.collect()
print('MEM: ' + humanfriendly.format_size(p.memory_info().rss))

fname = '../../data/benedict-xvi_curated_20190326.txt_preprocessed_en__disable(parser,ner,textcat)_.bin.bz2'

SpacyLang.factories['remove_whitespace_entities'] = lambda nlp, **cfg: remove_whitespace_entities

nlp = spacy.load('en_core_web_sm', disable=['parser','ner','textcat'])
nlp.tokenizer = keep_hyphen_tokenizer(nlp)

docs = read_spacy_docs(fname, nlp)
corpus = textacy.Corpus(docs=docs, lang=nlp)

print('MEM: ' + humanfriendly.format_size(p.memory_info().rss))

for doc in corpus[:25]:
    print('MEM: ' + humanfriendly.format_size(doc))


MEM: 1.31 GB
MEM: 1.31 GB
MEM: 1.34 GB


TypeError: unorderable types: Doc() >= int()